In [ ]:
import pandas as pd
import os

def average_service_time(file_path):
    """
    Reads the 'Settings' and 'Games' sheets of the combined.xlsx at file_path,
    identifies which side (host/guest) the UCLA player is in each match,
    sums the durations of only those games where that side served,
    and returns the average serve-time per match as an "M:SS" string.
    """
    # Infer player name from the parent folder
    player = os.path.basename(os.path.dirname(file_path))
    
    # Load sheets
    xls      = pd.ExcelFile(file_path)
    settings = xls.parse('Settings')
    games    = xls.parse('Games')
    
    # Clean up Settings: get one Host/Guest pair per match file
    settings_clean = (
        settings
        .groupby('__source_file__', as_index=False)
        .agg({'Host Team':'first', 'Guest Team':'first'})
    )
    
    # Collect total serve-time (in seconds) per match
    serve_totals = []
    for _, row in settings_clean.iterrows():
        match_file = row['__source_file__']
        host       = row['Host Team']
        guest      = row['Guest Team']
        
        # Determine which side UCLA player is
        if host  == player:
            side = 'host'
        elif guest == player:
            side = 'guest'
        else:
            # Player not in this match
            continue
        
        # Filter games for this match & side
        dfm = games[
            (games['__source_file__'] == match_file) &
            (games['Server']           == side)
        ].copy()
        
        # Convert Duration to numeric, drop missing
        dfm['Duration'] = pd.to_numeric(dfm['Duration'], errors='coerce')
        dfm = dfm[dfm['Duration'].notna()]
        
        # Sum up total seconds for this match
        serve_totals.append(dfm['Duration'].sum())
    
    if not serve_totals:
        return None
    
    # Compute the average seconds, then format as M:SS
    avg_seconds = sum(serve_totals) / len(serve_totals)
    total = int(round(avg_seconds))
    mins, secs = divmod(total, 60)
    return f"{mins}:{secs:02d}"
    

# Example usage:
path = "/Users/eshaan/Desktop/Tennis Scouting/consulting-spring2025/data/mens/Rudy Quan/combined.xlsx"
avg = average_service_time(path)  
avg

'25:50'

### Cj Work

In [ ]:
xls = pd.ExcelFile(f'../../data/mens/{player_name}/combined.xlsx')
settings = xls.parse('Settings')
games    = xls.parse('Games')

settings
games

,Game,Set,Server,Host Set Score,Guest Set Score,Game Winner,Start Time,Video Time,Duration,__source_file__
0,1,3,host,4,6,draw,05:45:20,9.330000,NaN,RudyQuan_Cal_1_25_25.xlsx
1,1,1,host,0,0,host,22:48:27,NaN,217.710007,RudyQuan_DeaconThomas_Indiana.xlsx
2,2,1,guest,1,0,host,22:52:05,NaN,269.410004,RudyQuan_DeaconThomas_Indiana.xlsx
3,3,1,host,2,0,host,22:56:34,NaN,165.479996,RudyQuan_DeaconThomas_Indiana.xlsx
4,4,1,guest,3,0,guest,22:59:20,NaN,250.080002,RudyQuan_DeaconThomas_Indiana.xlsx
...,...,...,...,...,...,...,...,...,...,...
124,13,2,host,3,1,guest,22:58:08,3584.629883,348.619995,RudyQuan_USC_2_22_25.xlsx
125,14,2,guest,3,2,guest,23:03:56,3933.250000,200.449997,RudyQuan_USC_2_22_25.xlsx
126,15,2,host,3,3,host,23:07:17,4133.700195,234.410004,RudyQuan_USC_2_22_25.xlsx
127,16,2,guest,4,3,host,23:11:11,4368.109863,271.630005,RudyQuan_USC_2_22_25.xlsx


In [ ]:
games_cleaned = games[['Server', 'Duration']]
games_cleaned
avg_seconds = games_cleaned[games_cleaned['Server'] == 'host']['Duration'].mean() # Automatically coerces NA
avg_seconds

238.44215458461537

In [ ]:
total = int(round(avg_seconds))
mins, secs = divmod(total, 60)
f"{mins}:{secs:02d}"

'3:58'

In [ ]:
settings_clean = (
        settings
        .groupby('__source_file__', as_index=False)
        .agg({'Host Team':'first', 'Guest Team':'first'})
    )

settings_clean

,__source_file__,Host Team,Guest Team
0,RudyQuan_Cal_1_25_25.xlsx,Rudy Quan,Carl Emil Overbeck
1,RudyQuan_DeaconThomas_Indiana.xlsx,Rudy Quan,Deacon Thomas
2,RudyQuan_FloridaState_9_23_24.xlsx,Rudy Quan,Youcef Rihane
3,RudyQuan_Indiana_3_7_25.xlsx,Rudy Quan,Deacon Thomas
4,RudyQuan_MichiganState_4_13_25.xlsx,Rudy Quan,Ozan Baris
5,RudyQuan_Pepperdine_2_27_25.xlsx,Rudy Quan,Edward Winter
6,RudyQuan_Purdue_3_9_25.xlsx,Rudy Quan,Aleksa Krivokapic
7,RudyQuan_TAMU_1_25_25.xlsx,Rudy Quan,JC Roddick
8,RudyQuan_UMichigan_4_25_25.xlsx,Rudy Quan,Will Cooksey
9,RudyQuan_USC_2_22_25.xlsx,Rudy Quan,Peter Makk
